In [ ]:
# Setup dan Import
!pip install -q google-cloud-aiplatform
!pip install -q google-cloud-storage
!pip install -q scikit-learn
!pip install -q pandas numpy matplotlib seaborn joblib
!pip install -q statsmodels pmdarima xgboost lightgbm

import os
import pandas as pd
import numpy as np
from google.cloud import storage
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from datetime import timedelta
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [ ]:
# Setup project
PROJECT_ID = "your-project-id"
BUCKET = "your-bucket"
REGION = "your-region"

In [ ]:
# Load processed data
def load_data_from_gcs(bucket_name, blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    data_str = blob.download_as_string()
    return pd.read_csv(pd.StringIO(data_str.decode('utf-8')))

In [ ]:
# Load the time series data prepared in notebook 1
processed_data = load_data_from_gcs(BUCKET, 'processed/training_data_with_weather.csv')
splits_info = load_data_from_gcs(BUCKET, 'processed/time_series_splits.csv')

In [ ]:
# Convert timestamp to datetime
processed_data['timestamp'] = pd.to_datetime(processed_data['timestamp'])

# Make sure data is sorted by timestamp
processed_data = processed_data.sort_values('timestamp')

In [ ]:
# Base Model Class for Calibration
class AirQualityCalibrator:
    def __init__(self):
        self.parameters = ['pm25', 'pm10', 'o3', 'co', 'no2']
        self.models = {}
        
    def prepare_features(self, data):
        # Include weather parameters based on notebook 1
        features = data[[
            'pm25_sensor', 'pm10_sensor', 'o3_sensor', 
            'co_sensor', 'no2_sensor', 'temperature', 'humidity',
            'wind_direction', 'wind_speed', 'precipitation'
        ]]
        return features
    
    def train(self, X_train, y_train, param):
        print(f"Training model for {param}...")
        
        # Define parameter grid for GridSearchCV
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [10, 20, 30],
            'min_samples_split': [2, 5, 10]
        }
        
        # Initialize base model
        base_model = RandomForestRegressor(random_state=42)
        
        # Perform GridSearchCV
        grid_search = GridSearchCV(
            base_model, param_grid, cv=5, 
            scoring='neg_root_mean_squared_error',
            n_jobs=-1
        )
        
        grid_search.fit(X_train, y_train)
        
        print(f"Best parameters for {param}: {grid_search.best_params_}")
        self.models[param] = grid_search.best_estimator_
        
        # Get feature importances
        feature_importances = pd.DataFrame({
            'feature': X_train.columns,
            'importance': grid_search.best_estimator_.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print("\nTop 5 important features:")
        print(feature_importances.head())
        
        return feature_importances
        
    def evaluate(self, X_test, y_test, param):
        predictions = self.models[param].predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        r2 = r2_score(y_test, predictions)
        mae = mean_absolute_error(y_test, predictions)
        
        # Handle zeros in y_test to avoid division by zero in MAPE
        mape = mean_absolute_percentage_error(y_test, predictions) * 100
        
        return {
            'rmse': rmse,
            'r2': r2,
            'mae': mae,
            'mape': mape,
            'predictions': predictions
        }

In [ ]:
# Time Series Model Class
class TimeSeriesForecaster:
    def __init__(self, target_parameter='pm25'):
        self.target_parameter = target_parameter
        self.sarimax_model = None
        self.xgb_model = None
        self.lgb_model = None
        self.feature_scaler = StandardScaler()
        
    def prepare_features(self, df, target_col):
        """
        Prepare features for time series forecasting.
        Uses the time series features created in notebook 1.
        """
        # Select all time-related features and other relevant features
        feature_cols = [
            # Time features
            'hour', 'day', 'day_of_week', 'month', 'is_weekend',
            'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos',
            
            # Lag features
            f'{target_col}_lag1h', f'{target_col}_lag3h', f'{target_col}_lag6h', 
            f'{target_col}_lag12h', f'{target_col}_lag24h',
            
            # Rolling window features
            f'{target_col}_rolling_mean_3h', f'{target_col}_rolling_mean_6h',
            f'{target_col}_rolling_mean_12h', f'{target_col}_rolling_std_3h',
            f'{target_col}_rolling_std_12h',
            
            # Trend features
            f'{target_col}_diff_1h', f'{target_col}_diff_3h',
            
            # Weather features
            'wind_direction', 'wind_speed', 'temperature', 'precipitation',
            'wind_speed_lag3h', 'temperature_lag3h'
        ]
        
        # Drop rows with NaN values which can occur after creating lag features
        features = df[feature_cols].dropna()
        return features
    
    def train_sarimax(self, train_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 24)):
        """
        Train a SARIMAX model for time series forecasting.
        """
        print(f"Training SARIMAX model for {self.target_parameter}...")
        
        # If auto_arima is True, automatically find the best parameters
        if order == 'auto':
            print("Running auto_arima to find best parameters...")
            auto_model = auto_arima(
                train_data,
                seasonal=True,
                m=24,  # Daily seasonality (24 hours)
                d=1,
                D=1,
                start_p=0, start_q=0,
                max_p=3, max_q=3,
                max_P=2, max_Q=2,
                max_d=2, max_D=1,
                trace=True,
                error_action='ignore',
                suppress_warnings=True,
                stepwise=True
            )
            print(f"Best SARIMAX parameters: {auto_model.order}, {auto_model.seasonal_order}")
            order = auto_model.order
            seasonal_order = auto_model.seasonal_order
        
        # Train SARIMAX model with given parameters
        model = SARIMAX(
            train_data,
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        
        self.sarimax_model = model.fit(disp=False)
        print(f"SARIMAX model training completed")
        
        # Print model summary
        print(self.sarimax_model.summary())
        
        return self.sarimax_model
    
    def train_ml_models(self, X_train, y_train):
        """
        Train XGBoost and LightGBM models for time series forecasting.
        """
        print(f"Training ML models for {self.target_parameter}...")
        
        # Scale features
        X_train_scaled = self.feature_scaler.fit_transform(X_train)
        
        # XGBoost model
        xgb_params = {
            'objective': 'reg:squarederror',
            'n_estimators': 100,
            'max_depth': 5,
            'learning_rate': 0.1,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42
        }
        
        self.xgb_model = xgb.XGBRegressor(**xgb_params)
        self.xgb_model.fit(X_train_scaled, y_train)
        
        # LightGBM model
        lgb_params = {
            'objective': 'regression',
            'n_estimators': 100,
            'max_depth': 5,
            'learning_rate': 0.1,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42
        }
        
        self.lgb_model = lgb.LGBMRegressor(**lgb_params)
        self.lgb_model.fit(X_train_scaled, y_train)
        
        # Get feature importances from XGBoost
        feature_importances = pd.DataFrame({
            'feature': X_train.columns,
            'importance': self.xgb_model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print("\nTop 5 important features from XGBoost:")
        print(feature_importances.head())
        
        return {'xgb': self.xgb_model, 'lgb': self.lgb_model}
    
    def predict_sarimax(self, steps=24):
        """
        Generate forecasts using the SARIMAX model.
        """
        return self.sarimax_model.forecast(steps=steps)
    
    def predict_ml_models(self, X_test):
        """
        Generate predictions using ML models.
        """
        X_test_scaled = self.feature_scaler.transform(X_test)
        
        xgb_pred = self.xgb_model.predict(X_test_scaled)
        lgb_pred = self.lgb_model.predict(X_test_scaled)
        
        # Create ensemble prediction (average of both models)
        ensemble_pred = (xgb_pred + lgb_pred) / 2
        
        return {
            'xgb': xgb_pred,
            'lgb': lgb_pred,
            'ensemble': ensemble_pred
        }
    
    def evaluate_forecasts(self, y_true, y_pred, model_name):
        """
        Evaluate time series forecasts with specific metrics.
        """
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        
        # Calculate MAPE, handling zeros carefully
        non_zero_mask = y_true != 0
        mape = np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100
        
        # Time series specific metrics
        # Calculate autocorrelation of residuals
        residuals = y_true - y_pred
        acf_1 = sm.tsa.acf(residuals, nlags=1)[1]  # First-order autocorrelation
        
        # Durbin-Watson test (values close to 2 indicate no autocorrelation)
        dw_stat = sm.stats.stattools.durbin_watson(residuals)
        
        # Normality test of residuals
        _, p_value = stats.shapiro(residuals)
        
        print(f"\nEvaluation metrics for {model_name}:")
        print(f"RMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"MAPE: {mape:.4f}%")
        print(f"Autocorrelation (lag 1): {acf_1:.4f}")
        print(f"Durbin-Watson: {dw_stat:.4f}")
        print(f"Shapiro-Wilk p-value: {p_value:.4f}")
        
        return {
            'rmse': rmse,
            'mae': mae,
            'mape': mape,
            'acf_1': acf_1,
            'dw_stat': dw_stat,
            'residuals_normality_p': p_value
        }
    
    def plot_forecast_vs_actual(self, y_true, y_pred, timestamps, model_name):
        """
        Plot forecast vs actual values.
        """
        plt.figure(figsize=(12, 6))
        plt.plot(timestamps, y_true, label='Actual', marker='o')
        plt.plot(timestamps, y_pred, label=f'{model_name} Forecast', marker='x')
        plt.title(f'{self.target_parameter} - Actual vs {model_name} Forecast')
        plt.xlabel('Timestamp')
        plt.ylabel(f'{self.target_parameter} Value')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.grid(True)
        plt.show()
        
        # Plot residuals
        plt.figure(figsize=(12, 6))
        residuals = y_true - y_pred
        plt.plot(timestamps, residuals, marker='o')
        plt.axhline(y=0, color='r', linestyle='-')
        plt.title(f'{model_name} Forecast Residuals')
        plt.xlabel('Timestamp')
        plt.ylabel('Residuals')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.grid(True)
        plt.show()
        
        # Plot residuals distribution
        plt.figure(figsize=(10, 6))
        sns.histplot(residuals, kde=True)
        plt.title(f'{model_name} Residuals Distribution')
        plt.xlabel('Residual Value')
        plt.grid(True)
        plt.show()
        
        # Plot autocorrelation of residuals
        plt.figure(figsize=(12, 6))
        sm.graphics.tsa.plot_acf(residuals, lags=24, alpha=0.05)
        plt.title(f'Autocorrelation of {model_name} Residuals')
        plt.tight_layout()
        plt.show()

In [ ]:
# Perform traditional model calibration
def prepare_training_data(df):
    # Include weather parameters as in notebook 1
    features = df[[
        'pm25_sensor', 'pm10_sensor', 'o3_sensor', 
        'co_sensor', 'no2_sensor', 'temperature', 'humidity',
        'wind_direction', 'wind_speed', 'precipitation'
    ]]
    
    targets = df[[
        'pm25_reference', 'pm10_reference', 'o3_reference',
        'co_reference', 'no2_reference'
    ]]
    
    return train_test_split(features, targets, test_size=0.2, random_state=42)

print("Starting traditional model calibration...")
X_train, X_test, y_train, y_test = prepare_training_data(processed_data)

In [ ]:
# Train traditional calibration models
calibrator = AirQualityCalibrator()
calib_results = {}
feature_importance_dict = {}

for param in calibrator.parameters:
    # Train
    feature_importance = calibrator.train(
        X_train, 
        y_train[f'{param}_reference'],
        param
    )
    
    feature_importance_dict[param] = feature_importance
    
    # Evaluate
    calib_results[param] = calibrator.evaluate(
        X_test,
        y_test[f'{param}_reference'],
        param
    )

In [ ]:
# Plot calibration results
def plot_calibration_results(results, parameters):
    # Set up the figure
    n_cols = 3
    n_rows = (len(parameters) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    axes = axes.flatten() if hasattr(axes, 'flatten') else [axes]
    
    for idx, param in enumerate(parameters):
        if idx < len(axes):
            y_true = y_test[f'{param}_reference']
            y_pred = results[param]['predictions']
            
            # Scatter plot
            sns.scatterplot(x=y_true, y=y_pred, ax=axes[idx], alpha=0.5)
            
            # Add perfect prediction line
            min_val = min(y_true.min(), y_pred.min())
            max_val = max(y_true.max(), y_pred.max())
            axes[idx].plot([min_val, max_val], [min_val, max_val], 'r--')
            
            # Add metrics to the title
            axes[idx].set_title(f'{param} Calibration\nRMSE: {results[param]["rmse"]:.2f}\nR²: {results[param]["r2"]:.2f}\nMAE: {results[param]["mae"]:.2f}')
            axes[idx].set_xlabel('True Values')
            axes[idx].set_ylabel('Predicted Values')
    
    # Hide any unused subplots
    for idx in range(len(parameters), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

print("\nPlotting calibration results...")
plot_calibration_results(calib_results, calibrator.parameters)

In [ ]:
# Plot feature importances
def plot_feature_importances(feature_importance_dict, parameters):
    n_cols = 3
    n_rows = (len(parameters) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    axes = axes.flatten() if hasattr(axes, 'flatten') else [axes]
    
    for idx, param in enumerate(parameters):
        if idx < len(axes):
            # Get top 10 features
            top_features = feature_importance_dict[param].head(10)
            
            # Create horizontal bar plot
            sns.barplot(
                data=top_features, 
                y='feature', 
                x='importance', 
                ax=axes[idx]
            )
            
            axes[idx].set_title(f'Top 10 Features for {param}')
            axes[idx].set_xlabel('Importance')
            axes[idx].set_ylabel('Feature')
    
    # Hide any unused subplots
    for idx in range(len(parameters), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

print("\nPlotting feature importances...")
plot_feature_importances(feature_importance_dict, calibrator.parameters)

In [ ]:
# Time Series Forecasting - Focus on PM2.5 (most important pollutant)
print("\nStarting time series forecasting for PM2.5...")

In [ ]:
# Setup time series forecaster
ts_forecaster = TimeSeriesForecaster(target_parameter='pm25')

In [ ]:
# Prepare time series data
# Convert timestamp columns in splits_info to datetime
for col in ['train_start', 'train_end', 'test_start', 'test_end']:
    if col in splits_info.columns:
        splits_info[col] = pd.to_datetime(splits_info[col])

# Get the last split for demonstration
last_split = splits_info.iloc[-1]

In [ ]:
# Create train and test sets based on the last split
train_data = processed_data[
    (processed_data['timestamp'] >= last_split['train_start']) & 
    (processed_data['timestamp'] <= last_split['train_end'])
]

test_data = processed_data[
    (processed_data['timestamp'] >= last_split['test_start']) & 
    (processed_data['timestamp'] <= last_split['test_end'])
]

print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")

In [ ]:
# Train SARIMAX model
# First check stationarity
from statsmodels.tsa.stattools import adfuller

# Apply ADF test on PM2.5 data
print("\nChecking stationarity of PM2.5 time series...")
adf_result = adfuller(train_data['pm25_sensor'].dropna())
print(f'ADF Statistic: {adf_result[0]}')
print(f'p-value: {adf_result[1]}')
print('Critical Values:')
for key, value in adf_result[4].items():
    print(f'\t{key}: {value}')

# If not stationary, we might want to take differences
if adf_result[1] > 0.05:
    print("Series is not stationary. Consider differencing in SARIMAX.")
else:
    print("Series is stationary.")

# Train SARIMAX model on univariate PM2.5 data
print("\nTraining SARIMAX model...")
# Use a smaller subset for SARIMAX due to computational complexity
sarimax_train = train_data['pm25_sensor'].iloc[-7*24:]  # Last 7 days
sarimax_model = ts_forecaster.train_sarimax(
    sarimax_train, 
    order=(1, 1, 1),  # ARIMA component (p, d, q)
    seasonal_order=(1, 1, 1, 24)  # Seasonal component (P, D, Q, s) with 24-hour seasonality
)

In [ ]:
# Forecast using SARIMAX
n_steps = len(test_data)
print(f"\nForecasting {n_steps} steps ahead...")
sarimax_forecast = ts_forecaster.predict_sarimax(steps=n_steps)

In [ ]:
# Evaluate SARIMAX forecast
# Make sure we have the same number of test points as forecast points
actual_values = test_data['pm25_sensor'].iloc[:len(sarimax_forecast)]
timestamps = test_data['timestamp'].iloc[:len(sarimax_forecast)]

print("\nEvaluating SARIMAX forecast...")
sarimax_metrics = ts_forecaster.evaluate_forecasts(
    actual_values.values, 
    sarimax_forecast, 
    'SARIMAX'
)

# Plot SARIMAX results
ts_forecaster.plot_forecast_vs_actual(
    actual_values.values,
    sarimax_forecast,
    timestamps,
    'SARIMAX'
)

In [ ]:
# Save Models
def save_models(calibrator, ts_forecaster, bucket_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    
    for param, model in calibrator.models.items():
        # Save locally first
        local_path = f'/tmp/{param}_model.joblib'
        joblib.dump(model, local_path)
        
        # Upload to GCS
        blob = bucket.blob(f'models/{param}_model.joblib')
        blob.upload_from_filename(local_path)
        
        print(f"Saved model for {param}")

save_models(calibrator, BUCKET)